# AivkConfig 类用法演示

本笔记本将系统演示 `AivkConfig` 配置基类的典型用法，包括：加载、保存、热重载、回调注册、同步/异步上下文管理等。适合用于开发自定义配置类和理解其高级特性。

## 1. 导入依赖库

首先导入AivkConfig及其依赖的所有库，包括pydantic、watchdog、threading等。

In [ ]:
from aivk.api import AivkConfig
from logging import getLogger
# 日志初始化
logger = getLogger("aivk.config")

## 2. 定义AivkConfig类

略

## 3. 加载配置文件示例

演示如何通过 `AivkConfig.load(fs, file_name)` 从JSON文件加载配置。

In [ ]:
from pathlib import Path
import json

# 假设有一个fs对象，模拟etc目录
class FakeFS:
    def __init__(self, etc_path):
        self.etc = Path(etc_path)

# 实际上是：AivkFS
# fs = AivkFS.getFS() 参数id可选 "aivk"（默认）表示使用系统配置目录

# 自定义配置类
class MyConfig(AivkConfig):
    username: str = "default"
    age: int = 0

# 创建模拟fs和配置文件路径
test_etc = Path("./test_etc")
test_etc.mkdir(exist_ok=True)
fs = FakeFS(test_etc)
file_name = "my_config.json"

# 写入一个初始配置文件
default_data = {"username": "alice", "age": 18}
with (test_etc / file_name).open("w", encoding="utf-8") as f:
    json.dump(default_data, f, ensure_ascii=False, indent=2)

# 加载配置
config = MyConfig.load(fs, file_name)
print(config)

## 4. 保存配置文件示例

演示如何调用 `AivkConfig` 实例的 `save()` 方法将配置保存到JSON文件。

In [ ]:
# 修改配置并保存
config.username = "bob"
config.age = 25
config.save()

# 检查文件内容
with (test_etc / file_name).open("r", encoding="utf-8") as f:
    print(f.read())

## 5. 注册热重载回调示例

演示如何使用 `onReload` 装饰器注册配置变更时的回调函数。

In [ ]:
# 注册热重载回调
@config.onReload
def on_config_reload(cfg: MyConfig):
    print(f"配置已热重载: {cfg}")

## 6. 监听配置文件变更示例

演示如何调用 `watch()` 方法监听配置文件变更并自动触发回调。

In [ ]:
# 启动监听
observer = config.watch()

# 修改配置文件以触发回调（模拟外部编辑）
with (test_etc / file_name).open("w", encoding="utf-8") as f:
    json.dump({"username": "eve", "age": 30}, f, ensure_ascii=False, indent=2)

import time
time.sleep(1)  # 等待watchdog检测到变更

# 停止监听
config.stop_watch()
print("监听已停止")

## 7. 使用同步上下文管理器示例

演示如何通过 `with` 语句使用AivkConfig，实现自动保存和资源清理。

In [ ]:
# 方法一：
# 使用同步上下文管理器
with MyConfig.load(fs, file_name) as cfg:
    cfg.username = "charlie"
    cfg.age = 40
    print(f"上下文中: {cfg}")
# 退出with后自动保存

cfg_instance = MyConfig.load(fs, file_name)

# 方法二：
# 实例直接用
with cfg_instance as cfg:
    cfg.username = "dave"
    cfg.age = 35
    print(f"直接实例中: {cfg}")

## 8. 使用异步上下文管理器示例

演示如何通过 `async with` 语句使用AivkConfig，实现异步自动保存和资源清理。

In [ ]:
# 使用异步上下文管理器
# 特别注意！！！.open()！！！！！！！！！！！！！！！ .open().open().open().open().open()

async with MyConfig.load(fs, file_name).open() as cfg:
    cfg.username = "dave"
    cfg.age = 50
    print(f"异步上下文中: {cfg}")
# 运行异步示例

# 方法二：
# 实例直接用

async with cfg_instance.open() as cfg:
    cfg.username = "eve"
    cfg.age = 55
    print(f"直接实例异步中: {cfg}")

# 8. 完整示例代码

In [ ]:
import random
from aivk.api import AivkFS , AivkConfig

fs = AivkFS.getFS("aivk")  # 获取AivkFS实例

class MyConfig(AivkConfig):
    username: str = "TEST"
    age: int = 999


# 不必操心文件路径 fs规定了路径再 fs.etc / file_name.json
with MyConfig.load(fs, "my_config") as cfg:
    print(f"使用AivkFS加载配置: {cfg}")
    # 修改配置
    cfg.username = "ssss"+"".join(random.choices("abcdefghijklmnopqrstuvwxyz", k=5))
    cfg.age = 60
    # 配置文件路径：
    print(f"配置文件路径: {cfg._config_path}") # 供debug看看 是fs.etc / filename.json #type: ignore

print(f"自动保存，修改后的配置: {cfg}")


# 监听文件修改：
observer = cfg.watch()
# 注册热重载回调
@cfg.onReload
def onReload(cfg: MyConfig):
    print(f"热重载回调触发: {cfg}")
    
    cfg.reload() # 这样即可重新加载配置

# 加了缓冲机制，避免出错
# 注意 配置文件也需要使用正确的类型
